<a href="https://colab.research.google.com/github/amyush/FrequencyDomain_Watermarking/blob/main/watermarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [5]:
!git clone https://github.com/amyush/FrequencyDomain_Watermarking.git

Cloning into 'FrequencyDomain_Watermarking'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [3]:
import numpy as np
import cv2
import math
from PIL import Image

# **Custom Functions**

In [8]:
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

def binaryToString(a):
  return chr(binaryToInt(a))

def integerToBinary(a):
  return int(bin(a)[2:])

def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

def calculatePSNR(originalImage, encodedImage):
  error = np.sum((originalImage.astype('float') - encodedImage.astype('float')) ** 2)
  error /= float(originalImage.shape[0] * originalImage.shape[1]);
  if error == 0:
      return 100
  PIXEL_MAX = 255.0
  return 20 * math.log10(PIXEL_MAX / math.sqrt(error))

def _compute_frame_dct(data):

    num_frames = data.shape[0]
    num_channels = data.shape[-1]

    dct = np.zeros(shape=data.shape[:-1])

    for i in range(num_frames):
        frame = data[i, :]
        if num_channels == 3:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        else:
            gray = np.squeeze(frame)

        dct[i, :, :] = cv2.dct(gray)

    return dct 

# **Steganography**

## **Hiding the data**

In [ ]:
def encodingImage(imageName, key, message):
    imgPath ="FrequencyDomain_Watermarking/"
    inputImage = cv2.imread(imgPath + imageName, cv2.IMREAD_UNCHANGED);
    outputImage = inputImage
    delimeter = "#@#@#@"
    height= inputImage.shape[0]
    width = inputImage.shape[1]
    dim = 1
    if(len(inputImage.shape) > 2):
        dim = inputImage.shape[2]
    _compute_frame_dct(inputImage)

    return inputImage, outputImage

In [ ]:
# coverFileName = input("Enter the cover filename: (eg- Cover_1.png or Cover_2.png)")
# key = input("Enter the key:")
# hiddenImageName = input("Enter the name of the image to be hidden:")

coverFileName = 
key = 
hiddenImageName = 


inputImage, outputImage = encodingMessage(fileName, key, message)

plt.subplot(1,2,1),plt.imshow(cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)), plt.title("Input Image")
plt.subplot(1,2,2),plt.imshow(cv2.cvtColor(outputImage, cv2.COLOR_BGR2RGB)), plt.title("Output Image") 

In [ ]:

#   binKey = stringToBinary(key)
#   binMsg = stringToBinary(message + delimeter)
#   lenBinKey = len(binKey)
#   lenBinMsg = len(binMsg)

#   index = -1
#   count = 0
#   pixel = 0

#   for k in range(dim):
#     for i in range(0, height, 1):
#       for j in range(0, width, 1):
#         index = index + 1;
#         if((int)(binKey[index%lenBinKey]) == 1):
#           if(count < lenBinMsg):
#             if(dim == 1):
#               pixel = integerToBinary(inputImage[i][j])
#               newPixel = ((int)(pixel/10)*10) + (int)(binMsg[count])
#               outputImage[i][j] = binaryToInt(newPixel);
#             else:
#               pixel = integerToBinary(inputImage[i][j][k])   
#               newPixel = ((int)(pixel/10)*10) + (int)(binMsg[count])
#               outputImage[i][j][k] = binaryToInt(newPixel);
#             count = count + 1;


#   psnr = calculatePSNR(inputImage, outputImage)
#   cv2.imwrite(imgPath + "encodedImage.png", outputImage)
#   return psnr, inputImage, outputImage;